In [3]:
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.dummy import DummyClassifier
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import learning_curve


# Mount du drive où se situe le dataset
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

# PATH génétrique des fichiers
path = "/content/drive/MyDrive/Accidentsroute/data/"



Mounted at /content/drive/


In [4]:
# PATH génétrique des fichiers
df = pd.read_csv(path + "merged_data_2018_2021_for_model.csv", sep=',', index_col=0)
df.head()

,mois,lum,agg,int,atm,col,catr,circ,nbv,vosp,prof,plan,surf,infra,situ,catv,obs,obsm,choc,manv,place,catu,grav,sexe_usagers,trajet,secu1,nb_veh,weekend,heure,periode,age_usagers,age_conducteur,sexe_conducteur
Num_Acc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201800000001,1,1.0,1,3.0,1.0,3.0,3,2.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,1,3.0,1.0,2.0,1.0,2,0,15,Après-midi,75-plus,95.0,1.0
201800000001,1,1.0,1,3.0,1.0,3.0,3,2.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,1.0,0.0,2.0,1.0,15.0,1.0,1,1.0,1.0,2.0,1.0,2,0,15,Après-midi,55-64,63.0,1.0
201800000002,2,1.0,2,3.0,1.0,1.0,4,2.0,2.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1,1.0,1.0,2.0,1.0,1,0,10,Matin,75-plus,76.0,1.0
201800000003,3,1.0,2,3.0,1.0,1.0,3,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,3.0,1.0,2.0,1.0,1.0,1.0,1,3.0,1.0,2.0,2.0,1,1,12,Après-midi,35-44,36.0,1.0
201800000004,5,1.0,2,1.0,1.0,4.0,3,2.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,9.0,1.0,21.0,1.0,1,1.0,1.0,2.0,1.0,1,1,18,Soir,35-44,41.0,1.0


In [5]:
#  1.1 Séparation de la variable cible et des variables explicatives en 3 tableaux :
# cible, catégorielle et quantitative

V = df.drop(['grav','nb_veh','age_conducteur'], axis=1)
W = df[['nb_veh','age_conducteur']]
y = df['grav']

In [6]:
# 1.2 Encodage des variables du tableau V (get_dummies avec utilisation de l'argument "drop")

V = V.apply(lambda x: x.astype('object') if x.dtype == 'int64' or x.dtype == 'float64' else x)
V = pd.get_dummies(V, drop_first=True).astype('float64')
#V = pd.get_dummies(V, drop_first=True)
# print(V.info())
# V.head()

<ipython-input-6-6023c61f710a>:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  V = pd.get_dummies(V, drop_first=True).astype('float64')
<ipython-input-6-6023c61f710a>:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  V = pd.get_dummies(V, drop_first=True).astype('float64')
<ipython-input-6-6023c61f710a>:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  V = pd.get_dummies(V, drop_first=True).astype('float64')
<ipython-input-6-6023c61f710a>:4: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  V = pd.get_dummies(V, drop_first=True).astype('float64')
<ipython-input-6-6023c61f710

In [7]:
# 1.3 Concaténation des variables du tableau V et du tableau W

X =pd.concat([V,W], axis=1)
print(X.shape)
X.head()

(427779, 174)


,mois_2,mois_3,mois_4,mois_5,mois_6,mois_7,mois_8,mois_9,mois_10,mois_11,mois_12,lum_2.0,lum_3.0,agg_2,int_2.0,int_3.0,atm_2.0,atm_3.0,atm_4.0,atm_5.0,col_2.0,col_3.0,col_4.0,catr_2,catr_3,catr_4,catr_5,circ_2.0,circ_3.0,circ_4.0,nbv_1.0,nbv_2.0,nbv_3.0,nbv_4.0,nbv_5.0,nbv_6.0,nbv_7.0,nbv_8.0,nbv_9.0,vosp_1.0,vosp_2.0,vosp_3.0,prof_2.0,prof_3.0,prof_4.0,plan_2.0,plan_3.0,plan_4.0,surf_2.0,surf_3.0,infra_1.0,infra_2.0,infra_3.0,infra_4.0,infra_5.0,infra_6.0,infra_7.0,infra_8.0,infra_9.0,situ_1.0,situ_2.0,situ_3.0,situ_4.0,situ_5.0,situ_6.0,situ_8.0,catv_1.0,catv_2.0,catv_3.0,catv_4.0,catv_5.0,catv_6.0,obs_1.0,obs_2.0,obs_3.0,obs_4.0,obs_5.0,obs_6.0,obs_7.0,obs_8.0,obs_9.0,obs_10.0,obs_11.0,obs_12.0,obs_13.0,obs_14.0,obs_15.0,obs_16.0,obs_17.0,obsm_1.0,obsm_2.0,obsm_4.0,obsm_5.0,obsm_6.0,obsm_9.0,choc_1.0,choc_2.0,choc_3.0,choc_4.0,manv_1.0,manv_2.0,manv_3.0,manv_4.0,manv_5.0,manv_6.0,manv_7.0,manv_8.0,manv_9.0,manv_10.0,manv_11.0,manv_12.0,manv_13.0,manv_14.0,manv_15.0,manv_16.0,manv_17.0,manv_18.0,manv_19.0,manv_20.0,manv_21.0,manv_22.0,manv_23.0,manv_24.0,manv_25.0,manv_26.0,place_2.0,place_3.0,catu_2,catu_3,sexe_usagers_2.0,trajet_2.0,trajet_3.0,secu1_2.0,secu1_3.0,secu1_4.0,weekend_1,heure_1,heure_2,heure_3,heure_4,heure_5,heure_6,heure_7,heure_8,heure_9,heure_10,heure_11,heure_12,heure_13,heure_14,heure_15,heure_16,heure_17,heure_18,heure_19,heure_20,heure_21,heure_22,heure_23,periode_Matin,periode_Nuit,periode_Soir,age_usagers_14-17,age_usagers_18-24,age_usagers_25-34,age_usagers_35-44,age_usagers_45-54,age_usagers_55-64,age_usagers_65-74,age_usagers_75-plus,sexe_conducteur_1.0,sexe_conducteur_2.0,nb_veh,age_conducteur
Num_Acc,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201800000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2,95.0
201800000001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2,63.0
201800000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,76.0
201800000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0

In [8]:
# 1.4 Division des données en ensemble d'entrainement et de test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101, stratify=y)

In [9]:
# 1.5 Standardisation des données de W

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train[['nb_veh', 'age_conducteur']] = scaler.fit_transform(X_train[['nb_veh', 'age_conducteur']])
X_test[['nb_veh', 'age_conducteur']] = scaler.transform(X_test[['nb_veh', 'age_conducteur']])

In [10]:
# Création variable cible binaire
y_bin = y.replace([1, 2], 0) \
         .replace([3, 4], 1)
y_train_bin = y_train.replace([1, 2], 0) \
                         .replace([3, 4], 1)
y_test_bin = y_test.replace([1, 2], 0) \
                         .replace([3, 4], 1)

In [11]:
print(X_train.shape)
print(y_train.shape)
print(y_train_bin.shape)
print(X_test.shape)
print(y_test.shape)
print(y_test_bin.shape)


(385001, 174)
(385001,)
(385001,)
(42778, 174)
(42778,)
(42778,)


In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(174,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100)


In [19]:
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy:', accuracy)


1337/1337 [==============================] - 3s 2ms/step - loss: -359475584.0000 - accuracy: 0.4222
Accuracy: 0.4222263693809509


In [12]:
import tensorflow as tf

model2 = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(174,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train, y_train_bin, epochs=100)

Epoch 1/100
12032/12032 [==============================] - 50s 4ms/step - loss: 0.4787 - accuracy: 0.7683
Epoch 2/100
12032/12032 [==============================] - 46s 4ms/step - loss: 0.4653 - accuracy: 0.7756
Epoch 3/100
12032/12032 [==============================] - 45s 4ms/step - loss: 0.4622 - accuracy: 0.7774
Epoch 4/100
12032/12032 [==============================] - 45s 4ms/step - loss: 0.4604 - accuracy: 0.7783
Epoch 5/100
12032/12032 [==============================] - 44s 4ms/step - loss: 0.4594 - accuracy: 0.7788
Epoch 6/100
12032/12032 [==============================] - 45s 4ms/step - loss: 0.4585 - accuracy: 0.7796
Epoch 7/100
12032/12032 [==============================] - 44s 4ms/step - loss: 0.4579 - accuracy: 0.7798
Epoch 8/100
12032/12032 [==============================] - 45s 4ms/step - loss: 0.4574 - accuracy: 0.7801
Epoch 9/100
12032/12032 [==============================] - 46s 4ms/step - loss: 0.4570 - accuracy: 0.7801
Epoch 10/100
12032/12032 [====================

In [13]:
_, accuracy = model2.evaluate(X_test, y_test_bin)
print('Accuracy:', accuracy)

1337/1337 [==============================] - 4s 3ms/step - loss: 0.4621 - accuracy: 0.7755
Accuracy: 0.7754687070846558
